## 11-3 long sequence
- adding FC layer and stacking RNN

In [1]:
import torch
import torch.optim as optim
import numpy as np

In [2]:
#torch.manual_seed(0)

- input data shape : 170,10,25
- outputs : 170,10,25

170 : sentence를 sequence_length로 나눔      
10 : sequence_length     
25 : input size, hidden size

In [3]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [4]:
char_set = list(set(sentence)) # 25
char_dic = {c: i for i, c in enumerate(char_set)} # 25

In [5]:
dic_size = len(char_dic) # 25
hidden_size = len(char_dic) # 25
sequence_length = 10  # 임의
learning_rate = 0.1

In [6]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, ':' ,x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str]) 
    y_data.append([char_dic[c] for c in y_str]) 

x_one_hot = [np.eye(dic_size)[x] for x in x_data]

0 : if you wan -> f you want
1 : f you want ->  you want 
2 :  you want  -> you want t
3 : you want t -> ou want to
4 : ou want to -> u want to 
5 : u want to  ->  want to b
6 :  want to b -> want to bu
7 : want to bu -> ant to bui
8 : ant to bui -> nt to buil
9 : nt to buil -> t to build
10 : t to build ->  to build 
11 :  to build  -> to build a
12 : to build a -> o build a 
13 : o build a  ->  build a s
14 :  build a s -> build a sh
15 : build a sh -> uild a shi
16 : uild a shi -> ild a ship
17 : ild a ship -> ld a ship,
18 : ld a ship, -> d a ship, 
19 : d a ship,  ->  a ship, d
20 :  a ship, d -> a ship, do
21 : a ship, do ->  ship, don
22 :  ship, don -> ship, don'
23 : ship, don' -> hip, don't
24 : hip, don't -> ip, don't 
25 : ip, don't  -> p, don't d
26 : p, don't d -> , don't dr
27 : , don't dr ->  don't dru
28 :  don't dru -> don't drum
29 : don't drum -> on't drum 
30 : on't drum  -> n't drum u
31 : n't drum u -> 't drum up
32 : 't drum up -> t drum up 
33 : t drum up  ->  

In [7]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [8]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [9]:
net = Net(dic_size, hidden_size, 2)

In [10]:
outputs = net(X)
outputs.shape

torch.Size([170, 10, 25])

In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [12]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        # print(i, j, ''.join([char_set[t] for t in result]), loss.item())
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]

    print(i, ':' ,predict_str, '\n')

0 : ,,uuu,r,u,ruuuuu,u,uru,uru,uru,ru,ru,,ruu,,u,,r,,rru,urr,,uruuuu,,,rrutu,uru,uru,ru,ru,ut,tuur,uruu,ruru,u,tu,,uru,,uuu,r,,uruurruur,uruuuuu,,uuu,,rr,ruru,u,ruuuu,rutr,uu,urr,rutu 

1 : nn  nnnnn  tnnnnnnn nnnn nn   n nn   nnnnn       nnn      tnnnnn     tnnn  nn tnn nn n  ltntnnn tnn   tnn  nn   ttnn  n    tnnn  tn   tnnnnn  tnn tnnnnn      ttnnn   n nnnnnnnn  n 

2 : totttttttttttttttttttttttotttttttttotttttotototttttttttttttttttttttttttttttttttttttttttttttttttttttttotttttttttotttttttttttttttttttttttttttttttottttttttttttottottttttttttttttttttt 

3 :                                                                                                                                                                                     

4 :   hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhahhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh 

5 : rrrrrrr rrrrrr r rr r r r r r r r r r rr rrrrr rrr r r r  r r   r 

86 : m yor tont to cuild a ship, don't wuum tp peop'e to ether to collect wood and don't wssign them tosks and do  , b t wathe  toach ther to cong for the e dless immensity of thekeoac 

87 : m yor wont to cuild a ship, don't wuum tp peop'e together to colbect wood and don't wssign them tosks and dorl, b t wather toach them to cong for the e dless immensity of the eoac 

88 : g yor wont to cuild a ship, don't w um tp weop'e together to collect wood and won't wssign them tosks and dor , b t wather toach them to cong for the e dless iomensity op thereoac 

89 : o yor wont to loild a ship, don't d um tp peop e together to collelt wood and don't dssign them tosks and dor , bht wather toach them to long for the e dless iomensity ff the eoac 

90 : g yor wont to luild a ship, don't w um tp weople together to lollelt wood and won't whsign them tosks and worl, b t wather toach them to long for the eudless iomensity of the eoac 

91 : g yop wont to luild a ship, don't w um tp weople together to